In [1]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

'\nfunction ClickConnect(){\nconsole.log("Working"); \ndocument.querySelector("colab-toolbar-button#connect").click() \n}\nsetInterval(ClickConnect,60000)\n'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!cat /proc/meminfo

MemTotal:       26751700 kB
MemFree:        23948040 kB
MemAvailable:   25753872 kB
Buffers:           92596 kB
Cached:          1949416 kB
SwapCached:            0 kB
Active:           751048 kB
Inactive:        1731576 kB
Active(anon):     419424 kB
Inactive(anon):      364 kB
Active(file):     331624 kB
Inactive(file):  1731212 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              3652 kB
Writeback:             0 kB
AnonPages:        440840 kB
Mapped:           245384 kB
Shmem:               928 kB
Slab:             186040 kB
SReclaimable:     131788 kB
SUnreclaim:        54252 kB
KernelStack:        4688 kB
PageTables:         6236 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    13375848 kB
Committed_AS:    3072992 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1840 kB
AnonHugePages:   

In [4]:
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [5]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager")
BiocManager::install("GSAR")

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/BiocManager_1.30.10.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 40205 bytes (39 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [6]:
%%R
library(tidyverse)
library(MASS)
library(GSAR)
library(parallel)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

R[write to console]: ✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.2     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: 
Attaching package: ‘MASS’


R[write to console]: The following object is masked from ‘package:dplyr’:

    select


R[write to console]: Loading required package: igraph

R[write to console]: 
Attaching package: ‘igraph’


R[write to console]: The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


R[write to console]: The following objects are masked from ‘package:purrr’:

    compose, simplify


R[write to console]: The following object is masked from ‘package:tidyr’:

    crossing


R[w

In [7]:
%%R

# Simplex sample generator
simplex_gen  <- function(n, samples){
    simplex_out  <- matrix(, nrow = samples, ncol = n)
    for(r in c(1:samples)){
        sample  <- runif(n-1,0,1)  %>%  
                    sort(decreasing=FALSE)
        sample  <- append(0.0, sample)  %>% 
                    append(1.0)
        sample  <- sample[-1] - sample
        sample  <- sample[1:n]

        simplex_out[r,]  <- sample
     }
    return(simplex_out)
}

# Genberate hypercube
hyper_gen  <- function(n, samples){
    simplex_out  <- matrix(, nrow = samples, ncol = n)
    for(r in c(1:samples)){
        sample  <- runif(n,-1,1) 
        sample  <- sample[1:n]
        simplex_out[r,]  <- sample
     }
    return(simplex_out)
}

# MST for simplex
mst <- function(file_, nsamp=10000, perm=50){
    # Read files with samples
    df_  <- read_csv(file_)  %>%  
    dplyr::select(starts_with('x_'))  %>% 
    sample_n(size=nsamp, replace=FALSE)
    # Get dim of the read file
    n_rows  <- dim(df_)[1]
    n_cols  <- dim(df_)[2]
        if (n_rows > (nsamp-1)){
        simplex_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
        
        df_  <- df_  %>% 
                data.matrix()   %>% 
                unname()
        
        df_  <- rbind(df_, simplex_sample)  
        df_  <- df_ %>% 
                aperm(c(2,1)) 
        result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                         nperm=perm, pvalue.only=FALSE)
        z_df  <- tibble(dim=n_cols, z=result$statistic, 
                        p_value=result$p.value,
                             nsamp=nsamp)
        gc()
        return(z_df)
        
    }
    
}


In [8]:
%%R
path  <-  '/content/drive/My Drive/mhar_paper_samples_2/'
set.seed(5)
simplex_gen(5,10)
files <- list.files(path)
files  <- paste(path, files, sep='')

# Extract dimenions
m <- regexpr("%[0-9]+.csv", files, perl=TRUE)
dimension <- regmatches(files, m)
m <- regexpr("[0-9]+", dimension, perl=TRUE)
dimension <- regmatches(dimension, m)
dimension

#Extract figure
m <- regexpr("[0-9]+_[a-z]+%", files, perl=TRUE)
figure <- regmatches(files, m)
m <- regexpr("[a-z]+", figure, perl=TRUE)
figure <- regmatches(figure, m)
figure

# Extract seed
m <- regexpr("/[0-9]+_", files, perl=TRUE)
sed <- regmatches(files, m)
m <- regexpr("[0-9]+", sed, perl=TRUE)
sed <- regmatches(sed, m)

df_simulations <- tibble(file=files, n=dimension, figure=figure, seed=sed)

In [13]:
%%R


for(f in files){
  file_name <- files[1]
  perm <- 50
  n_sample <-5000
  print(file_name)
  df_  <- read_csv(file_name)
  df_ <- df_[c(1:n_sample),]
  # Get dim of the read file
  n_rows  <- dim(df_)[1]
  n_cols  <- dim(df_)[2]
  print(n_rows)
  print(n_cols)

  if (grepl('hyper', files[1], fixed = TRUE)){
    real_sample  <- suppressWarnings(hyper_gen(n_cols,n_rows))
  }
  if (grepl('simplex', files[1], fixed = TRUE)){
    real_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
  }

  df_  <- df_  %>% data.matrix()   %>% unname()
          
  df_  <- rbind(df_, real_sample)  
  df_  <- df_ %>% aperm(c(2,1)) 
  dim(df_)
  result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                          nperm=2, pvalue.only=FALSE)
  z_df  <- tibble(dim=n_cols, z=result$statistic, 
                          p_value=result$p.value,
                              nsamp=n_sample)
  gc()   
}




[1] "/content/drive/My Drive/mhar_paper_samples_2/100_hypercube%1000%3.csv"


R[write to console]: Parsed with column specification:
cols(
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double()
)



[1] 5000
[1] 3
            used   (Mb) gc trigger   (Mb)   max used    (Mb)
Ncells   1019966   54.5   40954045 2187.2   53689577  2867.4
Vcells 285352864 2177.1 1070634692 8168.3 1338293364 10210.4


In [15]:
%%R
result
z_df

# A tibble: 1 x 4
    dim      z p_value nsamp
  <int>  <dbl>   <dbl> <dbl>
1     3 -0.798   0.333  5000
